In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import time

df = pd.read_csv('../csv/NOW_dataframe_cleaned.csv', index_col=0)
gridFeatures = pd.read_csv('../csv/gridFeatures.csv', index_col=0)
localityFeatures = pd.read_csv('../csv/localityFeatures.csv', index_col=0)
timeUnits = ['old','MN1','MN2','MN3','MN4','MN5','MN6','MN7-8','MN9','MN10','MN11','MN12','MN13','MN14','MN15','MN16','MN17','MQ18','MQ19','recent']

In [2]:
# USING ONLY LOCALITIES IN EUROPE AND CLOSE BY
localityFeatures = localityFeatures[(localityFeatures['LAT']>35) & (localityFeatures['LONG']<40)]
# original gridFeatures has resolution 0.1X0.1 degrees, here we go to a 0.5X0.1 resolution to save 
# computation time
gridFeatures = gridFeatures.loc[::5]
display(gridFeatures.head())

,LONG,LAT
0,-6.3,35.0
5,-5.8,35.0
10,-5.3,35.0
15,-4.8,35.0
20,-4.3,35.0


In [3]:
gridFeatures_MN1 = gridFeatures.copy()
gridFeatures_MN2 = gridFeatures.copy()
gridFeatures_MN3 = gridFeatures.copy()
gridFeatures_MN4 = gridFeatures.copy()
gridFeatures_MN5 = gridFeatures.copy()
gridFeatures_MN6 = gridFeatures.copy()
gridFeatures_MN7 = gridFeatures.copy()
gridFeatures_MN9 = gridFeatures.copy()
gridFeatures_MN10 = gridFeatures.copy()
gridFeatures_MN11 = gridFeatures.copy()
gridFeatures_MN12 = gridFeatures.copy()
gridFeatures_MN13 = gridFeatures.copy()
gridFeatures_MN14 = gridFeatures.copy()
gridFeatures_MN15 = gridFeatures.copy()
gridFeatures_MN16 = gridFeatures.copy()
gridFeatures_MN17 = gridFeatures.copy()
gridFeatures_MQ18 = gridFeatures.copy()
gridFeatures_MQ19 = gridFeatures.copy()

gridFeaturesAll = [gridFeatures_MN1,gridFeatures_MN2,gridFeatures_MN3,gridFeatures_MN4,gridFeatures_MN5,\
                   gridFeatures_MN6,gridFeatures_MN7,gridFeatures_MN9,gridFeatures_MN10,gridFeatures_MN11,\
                   gridFeatures_MN12,gridFeatures_MN13,gridFeatures_MN14,gridFeatures_MN15,gridFeatures_MN16,\
                   gridFeatures_MN17,gridFeatures_MQ18,gridFeatures_MQ19]

In [4]:
# HYP and LOP data from these orders can be used for environmental estimation
permissibleOrders = ['Perissodactyla','Artiodactyla','Primates','Proboscidea','Hyracoidea','Dinocerata',\
                     'Embrithopoda','Notoungulata','Astrapotheria','Pyrotheria','Litopterna']
print('NUMBER OF OCCURRENCES (includes "old" and "recent" time units)')
print()
print('Perissodactyla: ' + str(df[df['ORDER']=='Perissodactyla'].shape[0]))
print('Artiodactyla: ' + str(df[df['ORDER']=='Artiodactyla'].shape[0]))
print('Primates: ' + str(df[df['ORDER']=='Primates'].shape[0]))
print('Proboscidea: ' + str(df[df['ORDER']=='Proboscidea'].shape[0]))
print('Hyracoidea: ' + str(df[df['ORDER']=='Hyracoidea'].shape[0]))
print('Dinocerata: ' + str(df[df['ORDER']=='Dinocerata'].shape[0]))
print('Embrithopoda: ' + str(df[df['ORDER']=='Embrithopoda'].shape[0]))
print('Notoungulata: ' + str(df[df['ORDER']=='Notoungulata'].shape[0]))
print('Astrapotheria: ' + str(df[df['ORDER']=='Astrapotheria'].shape[0]))
print('Pyrotheria: ' + str(df[df['ORDER']=='Pyrotheria'].shape[0]))
print('Litopterna: ' + str(df[df['ORDER']=='Litopterna'].shape[0]))

NUMBER OF OCCURRENCES (includes "old" and "recent" time units)

Perissodactyla: 4968
Artiodactyla: 9533
Primates: 1748
Proboscidea: 1933
Hyracoidea: 141
Dinocerata: 32
Embrithopoda: 6
Notoungulata: 103
Astrapotheria: 9
Pyrotheria: 2
Litopterna: 41


In [5]:
# PRE-ALLOCATION
for grid in gridFeaturesAll:
    grid['FIRST OCCURRENCES IN FOCAL AREA (t)'] = 0
    grid['LAST OCCURRENCES IN FOCAL AREA (t)'] = 0
    grid['OCCURRENCES IN FOCAL AREA (t)'] = 0
    grid['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t)'] = 0
    grid['HYP1'] = 0
    grid['HYP2'] = 0
    grid['HYP3'] = 0
    grid['LOP0'] = 0
    grid['LOP1'] = 0
    grid['LOP2'] = 0

gridFeatures_MN15.head()

,LONG,LAT,FIRST OCCURRENCES IN FOCAL AREA (t),LAST OCCURRENCES IN FOCAL AREA (t),OCCURRENCES IN FOCAL AREA (t),UNWEIGHTED OCCURRENCES IN FOCAL AREA (t),HYP1,HYP2,HYP3,LOP0,LOP1,LOP2
0,-6.3,35.0,0,0,0,0,0,0,0,0,0,0
5,-5.8,35.0,0,0,0,0,0,0,0,0,0,0
10,-5.3,35.0,0,0,0,0,0,0,0,0,0,0
15,-4.8,35.0,0,0,0,0,0,0,0,0,0,0
20,-4.3,35.0,0,0,0,0,0,0,0,0,0,0


In [6]:
d = 10 # here we use a square around the focal locality with 2*d degree sides to find candidate localities
radius = 500 # radius (km) of the circle around the focal grid point; we look for localities within this circle

timeUnit_count = 1
for gridFeatures in gridFeaturesAll:
    tic = time.clock()
    count = 0
    for ix in gridFeatures.index: 
        focalTimeUnit = timeUnits[timeUnit_count]
        focalTimeUnitIndex = timeUnits.index(focalTimeUnit)
        focalLONG = gridFeatures.loc[ix,'LONG'] 
        focalLAT = gridFeatures.loc[ix,'LAT'] 
        
        # calculate occurrences at t: we can take focal localities with timeunit MN1 - MQ19
        if (focalTimeUnitIndex >= timeUnits.index('MN1')) & (focalTimeUnitIndex <= timeUnits.index('MQ19')):         
            # we look for candidate localities in the focal area 
            candidateLocalities = localityFeatures[(localityFeatures['TIMEUNIT']==timeUnits[focalTimeUnitIndex]) & (localityFeatures['LONG']>focalLONG-d) & (localityFeatures['LONG']<focalLONG+d) & (localityFeatures['LAT']>focalLAT-d) & (localityFeatures['LAT']<focalLAT+d)]
            for ix_candidate in candidateLocalities.index:
                distance = geodesic(localityFeatures.loc[ix_candidate,['LAT','LONG']], gridFeatures.loc[ix,['LAT','LONG']]).km
                if distance < radius:
                    weight = (radius - distance)/radius
                    gridFeatures.loc[ix,'FIRST OCCURRENCES IN FOCAL AREA (t)'] += weight * candidateLocalities.loc[ix_candidate,'NUMBER OF FIRST OCCURRENCES']
                    gridFeatures.loc[ix,'LAST OCCURRENCES IN FOCAL AREA (t)'] += weight * candidateLocalities.loc[ix_candidate,'NUMBER OF LAST OCCURRENCES']
                    gridFeatures.loc[ix,'OCCURRENCES IN FOCAL AREA (t)'] += weight * candidateLocalities.loc[ix_candidate,'NUMBER OF OCCURRENCES']
                    gridFeatures.loc[ix,'UNWEIGHTED OCCURRENCES IN FOCAL AREA (t)'] += candidateLocalities.loc[ix_candidate,'NUMBER OF OCCURRENCES']
                    
                    # environmental estimation
                    occurrences = df.loc[(df['LIDNUM']==ix_candidate) & (df['ORDER'].isin(permissibleOrders))]
                    for ix_occurrence in occurrences.index:
                        if occurrences.loc[ix_occurrence,'HYP'] == 1:
                            gridFeatures.loc[ix,'HYP1'] += weight
                        elif occurrences.loc[ix_occurrence,'HYP'] == 2:
                            gridFeatures.loc[ix,'HYP2'] += weight
                        elif occurrences.loc[ix_occurrence,'HYP'] == 3:
                            gridFeatures.loc[ix,'HYP3'] += weight
                        
                        if occurrences.loc[ix_occurrence,'LOP'] == 0:
                            gridFeatures.loc[ix,'LOP0'] += weight
                        elif occurrences.loc[ix_occurrence,'LOP'] == 1:
                            gridFeatures.loc[ix,'LOP1'] += weight
                        elif occurrences.loc[ix_occurrence,'LOP'] == 2:
                            gridFeatures.loc[ix,'LOP2'] += weight       
        
        # show run progress
        count += 1
        if count % 10000 == 0:
            toc = time.clock()
            print('Progress: ' + str(count) + '/' + str(gridFeatures.shape[0]))
            print('Time elapsed: ' + str((toc-tic)/60) + ' minutes.')
        
    toc = time.clock() 
    print('Progress: ' + str(gridFeatures.shape[0]) + '/' + str(gridFeatures.shape[0]))
    print('Time elapsed: ' + str((toc-tic)/60) + ' minutes.')         
    display(focalTimeUnit)
    display(gridFeatures.head())
    timeUnit_count += 1

Progress: 10000/15988
Time elapsed: 8.15216165 minutes.
Progress: 15988/15988
Time elapsed: 9.606827933333333 minutes.


'MN1'

,LONG,LAT,FIRST OCCURRENCES IN FOCAL AREA (t),LAST OCCURRENCES IN FOCAL AREA (t),OCCURRENCES IN FOCAL AREA (t),UNWEIGHTED OCCURRENCES IN FOCAL AREA (t),HYP1,HYP2,HYP3,LOP0,LOP1,LOP2
0,-6.3,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
5,-5.8,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
10,-5.3,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
15,-4.8,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
20,-4.3,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0


Progress: 10000/15988
Time elapsed: 38.37778961666666 minutes.
Progress: 15988/15988
Time elapsed: 45.502075500000004 minutes.


'MN2'

...

'MQ18'

,LONG,LAT,FIRST OCCURRENCES IN FOCAL AREA (t),LAST OCCURRENCES IN FOCAL AREA (t),OCCURRENCES IN FOCAL AREA (t),UNWEIGHTED OCCURRENCES IN FOCAL AREA (t),HYP1,HYP2,HYP3,LOP0,LOP1,LOP2
0,-6.3,35.0,2.130800,5.741085,10.636515,65.0,0.787013,0.147892,3.302013,2.293161,0.311635,1.869811
5,-5.8,35.0,3.170450,8.092443,14.725434,65.0,1.049234,0.279309,4.197134,2.728936,0.442848,2.657087
10,-5.3,35.0,4.125012,10.280947,18.486064,66.0,1.288773,0.401373,4.995377,3.137533,0.563382,3.380291
15,-4.8,35.0,5.126633,12.509331,22.233513,92.0,1.524769,0.511777,5.762722,3.594368,0.670250,4.073092
20,-4.3,35.0,6.528144,15.223256,26.639820,92.0,1.812381,0.607741,6.587921,4.079090,0.759822,4.840245


Progress: 10000/15988
Time elapsed: 33.42641851666667 minutes.
Progress: 15988/15988
Time elapsed: 44.771550616666595 minutes.


'MQ19'

,LONG,LAT,FIRST OCCURRENCES IN FOCAL AREA (t),LAST OCCURRENCES IN FOCAL AREA (t),OCCURRENCES IN FOCAL AREA (t),UNWEIGHTED OCCURRENCES IN FOCAL AREA (t),HYP1,HYP2,HYP3,LOP0,LOP1,LOP2
0,-6.3,35.0,2.002215,3.432750,4.493463,20.0,1.060713,0.238422,0.476845,1.060713,0.119211,0.596056
5,-5.8,35.0,2.222888,4.445984,5.650057,20.0,1.204073,0.370516,0.741032,1.204073,0.185258,0.926290
10,-5.3,35.0,2.481006,5.533281,6.825189,25.0,1.291908,0.492760,0.985521,1.291908,0.246380,1.231901
15,-4.8,35.0,2.705606,6.540326,7.879271,25.0,1.338945,0.602565,1.205129,1.338945,0.301282,1.506412
20,-4.3,35.0,2.865728,7.379244,8.736664,25.0,1.357420,0.696777,1.393555,1.357420,0.348389,1.741943


In [7]:
# calculate additional stats

# gridFeatures_MN1
gridFeatures = gridFeaturesAll[0]
# t-1
gridFeatures['OCCURRENCES IN FOCAL AREA (t-1)'] = 0
gridFeatures['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t-1)'] = 0
# t+1
gridFeatures['OCCURRENCES IN FOCAL AREA (t+1)'] = gridFeaturesAll[1]['OCCURRENCES IN FOCAL AREA (t)']
gridFeatures['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t+1)'] = gridFeaturesAll[1]['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t)']
# t
gridFeatures['PROPORTION OF FIRST OCCURRENCES IN FOCAL AREA (t)'] = gridFeatures['FIRST OCCURRENCES IN FOCAL AREA (t)'] / gridFeatures['OCCURRENCES IN FOCAL AREA (t)']
gridFeatures['PROPORTION OF LAST OCCURRENCES IN FOCAL AREA (t)'] = gridFeatures['LAST OCCURRENCES IN FOCAL AREA (t)'] / gridFeatures['OCCURRENCES IN FOCAL AREA (t)']
gridFeatures['MEAN HYP'] = (gridFeatures['HYP1']*1 + gridFeatures['HYP2']*2 + gridFeatures['HYP3']*3) / (gridFeatures['HYP1'] + gridFeatures['HYP2'] + gridFeatures['HYP3']) 
gridFeatures['MEAN LOP'] = (gridFeatures['LOP1']*1 + gridFeatures['LOP2']*2) / (gridFeatures['LOP0'] + gridFeatures['LOP1'] + gridFeatures['LOP2']) 

# gridFeatures_MN2 - gridFeatures_MQ18
count = 1
for gridFeatures in gridFeaturesAll[1:17]:
    gridFeatures['OCCURRENCES IN FOCAL AREA (t-1)'] = gridFeaturesAll[count-1]['OCCURRENCES IN FOCAL AREA (t)']
    gridFeatures['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t-1)'] = gridFeaturesAll[count-1]['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t)']
    gridFeatures['OCCURRENCES IN FOCAL AREA (t+1)'] = gridFeaturesAll[count+1]['OCCURRENCES IN FOCAL AREA (t)']
    gridFeatures['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t+1)'] = gridFeaturesAll[count+1]['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t)']
    gridFeatures['PROPORTION OF FIRST OCCURRENCES IN FOCAL AREA (t)'] = gridFeatures['FIRST OCCURRENCES IN FOCAL AREA (t)'] / gridFeatures['OCCURRENCES IN FOCAL AREA (t)']
    gridFeatures['PROPORTION OF LAST OCCURRENCES IN FOCAL AREA (t)'] = gridFeatures['LAST OCCURRENCES IN FOCAL AREA (t)'] / gridFeatures['OCCURRENCES IN FOCAL AREA (t)']
    gridFeatures['MEAN HYP'] = (gridFeatures['HYP1']*1 + gridFeatures['HYP2']*2 + gridFeatures['HYP3']*3) / (gridFeatures['HYP1'] + gridFeatures['HYP2'] + gridFeatures['HYP3']) 
    gridFeatures['MEAN LOP'] = (gridFeatures['LOP1']*1 + gridFeatures['LOP2']*2) / (gridFeatures['LOP0'] + gridFeatures['LOP1'] + gridFeatures['LOP2']) 
    count += 1
    
# gridFeatures_MQ19
gridFeatures = gridFeaturesAll[17]
gridFeatures['OCCURRENCES IN FOCAL AREA (t-1)'] = gridFeaturesAll[16]['OCCURRENCES IN FOCAL AREA (t)']
gridFeatures['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t-1)'] = gridFeaturesAll[16]['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t)']
gridFeatures['OCCURRENCES IN FOCAL AREA (t+1)'] = 0
gridFeatures['UNWEIGHTED OCCURRENCES IN FOCAL AREA (t+1)'] = 0
gridFeatures['PROPORTION OF FIRST OCCURRENCES IN FOCAL AREA (t)'] = gridFeatures['FIRST OCCURRENCES IN FOCAL AREA (t)'] / gridFeatures['OCCURRENCES IN FOCAL AREA (t)']
gridFeatures['PROPORTION OF LAST OCCURRENCES IN FOCAL AREA (t)'] = gridFeatures['LAST OCCURRENCES IN FOCAL AREA (t)'] / gridFeatures['OCCURRENCES IN FOCAL AREA (t)']    
gridFeatures['MEAN HYP'] = (gridFeatures['HYP1']*1 + gridFeatures['HYP2']*2 + gridFeatures['HYP3']*3) / (gridFeatures['HYP1'] + gridFeatures['HYP2'] + gridFeatures['HYP3']) 
gridFeatures['MEAN LOP'] = (gridFeatures['LOP1']*1 + gridFeatures['LOP2']*2) / (gridFeatures['LOP0'] + gridFeatures['LOP1'] + gridFeatures['LOP2']) 

In [8]:
# environmental estimation

for gridFeatures in gridFeaturesAll:
    gridFeatures['MAT'] = 24.7 + 13.8*gridFeatures['MEAN HYP']-25.1*gridFeatures['MEAN LOP']
    gridFeatures['MAP'] = 2727.7 - 411.9*gridFeatures['MEAN HYP']-859.7*gridFeatures['MEAN LOP']
    gridFeatures['NPP'] = 2957.8 - 304.3*gridFeatures['MEAN HYP']-1043.7*gridFeatures['MEAN LOP']

In [10]:
# save data

timeUnit_count = 1
for gridFeatures in gridFeaturesAll:
    focalTimeUnit = timeUnits[timeUnit_count]
    filename = '../csv/gridFeatures500/gridFeatures_' + focalTimeUnit + '.csv'
    gridFeatures.to_csv(filename)
    timeUnit_count += 1